# 5 Pytorch Custom Data set

In [ ]:
import pandas as pd
import numpy as np
import torch as tc
from torch import nn
# import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

## 5.1 Getting our datasets and becoming one with Data

In [ ]:
import os
def walk_through_dir(dir_path):
  """walks through dir path returning its content"""
  for dirpath,dirnames,filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}")

In [ ]:
walk_through_dir(dir_path="DATA")

In [ ]:
from pathlib import Path
data_path = Path("DATA/")

In [ ]:
# setting up our train and test path
train_dir = data_path / "train"
test_dir = data_path / "test"
train_dir,test_dir

### 5.1.1 Visuallising our image

In [ ]:
import random
from PIL import Image

# set seed
tc.manual_seed(42)

# 1. get all the image path
image_path_list = list(data_path.glob("*/*/*.jpg"))
# image_path_list

# 2. Plot random image
random_image_path = random.choice(image_path_list)
print(random_image_path)

# 3. get the image class
image_class = random_image_path.parent.stem
print(image_class)

# 4. open the image
img = Image.open(random_image_path)

# 5. print meta data
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image Height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
plt.imshow(img)
plt.axis(False)

In [ ]:
img_as_array = np.asarray(img)
img_as_array

## 5.2 Transforming our data

### 5.2.1 Transform data with `torchvision.transform`

In [ ]:
data_transform = transforms.Compose([
  # resize the image
  transforms.Resize(size=(64,64)),
  # flip the image
  transforms.RandomHorizontalFlip(p=0.5),
  # Turn image into a torch tensor
  ToTensor()
])

data_transform(img).shape

In [ ]:
def plot_transformed_images(images_paths,transform,n=3,seed=None):
  """
    Selects random images from a path of images and loads/transforms them then plots the original vs the transformed version
  """
  if seed:
    random.seed(seed)
    random_image_paths = random.sample(images_paths,k=n)
    for image_path in random_image_paths:
      with Image.open(image_path) as f:
        fig,ax = plt.subplots(nrows=1,ncols=2)
        ax[0].imshow(f)
        ax[0].set_title(f"Origin\nSize: {f.size}")
        ax[0].axis(False)
        
        # transform and plot target image
        transformed_image = transform(f).permute(1,2,0) # not we will need to change the sahpe for the transform
        ax[1].imshow(transformed_image)
        ax[1].set_title(f"Transformed\nShape: {transformed_image.shape}")
        ax[1].axis(False)
        
        fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16) 

plot_transformed_images(image_path_list,transform=data_transform,seed=42)

## 5.3 Option1 Loading image data using ImageFolder

In [ ]:
# use image folder to create datasets
train_data = datasets.ImageFolder(root=train_dir,transform=data_transform,target_transform=None)
test_data = datasets.ImageFolder(root=test_dir,transform=data_transform,target_transform=None)

train_data,test_data

In [ ]:
# getting classnames
train_classnames = train_data.classes
train_classnames

In [ ]:
# Index on the train data datasets
img,label =  train_data[1][0], train_data[1][1]
# img,label
print(f"Image Tensor:\n{img}") 
print(f"Image Shape:\n{img.dtype}") 
print(f"Image Datatype:\n{img.dtype}")
print(f"Image Label:\n{train_classnames[label]}")

In [ ]:
# Rearrange the order of dimension
img_permute = img.permute(1,2,0)

# print out different shapes
print(f"Original shape: {img.shape} -> [color_channels, height, width]")
print(f"Image Permute shape: {img_permute.shape} -> [height, width, color_channels]")

# plot the image
plt.figure(figsize=(10,7))
plt.imshow(img_permute)
plt.axis(False)
plt.title(train_classnames[label],fontsize=14)

### 5.3.1 Turn loaded images into Dataloader

In [ ]:
import os
os.cpu_count()

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 1
train_dataloader = DataLoader(
    dataset=train_data, batch_size=BATCH_SIZE, num_workers=os.cpu_count(), shuffle=True
)
test_dataloader = DataLoader(
    dataset=test_data, batch_size=BATCH_SIZE, num_workers=os.cpu_count(), shuffle=True
)

len(train_dataloader),len(test_dataloader)

## 5.4 Option2 Loading Image Data with a custom `Dataset`

In [ ]:
from typing import Tuple,Dict,List
train_data.classes, train_data.class_to_idx

### 5.4.1 Creating a helper function to get classnames

In [ ]:
# def 
# setup target directory
target_directory = train_dir
print(f"Target dir: {target_directory}")

# get the class names from the target directory
class_names_found = sorted([entry.name for entry in list(os.scandir(target_directory))])
class_names_found

In [ ]:
def find_classes(directory:str) -> Tuple[List[str],Dict[str,int]]:
  """Finds the class folder names in a target directory."""
  # 1. Get the class names by scanning the target directory
  classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
  
  # 2. raise an error if class names nt found
  if not classes:
    raise FileNotFoundError(f"Couldn't find any classes in {directory}...please check the file structure")
  
  # 3. create a dictionary of index labels
  classes_to_idx = {class_name: i for i, class_name in enumerate(classes)}
  return classes, classes_to_idx

In [ ]:
find_classes(target_directory)